In [1]:
import numpy as np
import tensorflow as tf

/home/train/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#sort of hacky, will make better later

import pandas as pd

data = pd.read_csv('KDDTest+.csv', header=None)

data_temp = data[[x for x in range(4,41)]]
y_data_temp = data[2]

In [3]:
data[[x for x in range(19,25)]][0:5]

,19,20,21,22,23,24
0,0,0,0,229,10,0.0
1,0,0,0,136,1,0.0
2,0,0,0,1,1,0.0
3,0,0,0,1,65,0.0
4,0,0,0,1,8,0.0


In [4]:
xdata = data_temp.values
k = 6
num_steps = 21
batch_size = 1024
num_features = 37 #41

X = tf.placeholder(tf.float32, shape=[None, num_features])
Y = tf.placeholder(tf.float32, shape=[None, num_classes])

kmeans = tf.contrib.factorization.KMeans(inputs=X, num_clusters=k, distance_metric='cosine',
                use_mini_batch=True)


# Build KMeans graph
#tensorflow 1.4+ should add cluster_centers_var as the 5th value
(all_scores, cluster_idx, scores, cluster_centers_initialized, 
init_op, train_op) = kmeans.training_graph()


Instructions for updating:
dim is deprecated, use axis instead


In [5]:
cluster_idx = cluster_idx[0] #not sure why this works
avg_distance = tf.reduce_mean(scores)

In [6]:
avg_distance = tf.reduce_mean(scores)

init_vars = tf.global_variables_initializer()

sess = tf.Session()

sess.run(init_vars, feed_dict={X: xdata})
sess.run(init_op, feed_dict={X: xdata})

for i in range(1, num_steps + 1):
    _, d, idx = sess.run([train_op, avg_distance, cluster_idx],
                         feed_dict={X: xdata})
    if i % 10 == 0 or i == 1:
        print("Step %i, Avg Distance: %f" % (i, d))

Step 1, Avg Distance: 0.231318
Step 10, Avg Distance: 0.060416
Step 20, Avg Distance: 0.057914


In [7]:
for i in range(0, len(data_temp[0:10])):
    cluster_index = idx[i]
    print('point:', i, ' is in cluster ', cluster_index)

point: 0  is in cluster  5
point: 1  is in cluster  5
point: 2  is in cluster  2
point: 3  is in cluster  1
point: 4  is in cluster  1
point: 5  is in cluster  4
point: 6  is in cluster  3
point: 7  is in cluster  1
point: 8  is in cluster  4
point: 9  is in cluster  4
